<a href="https://colab.research.google.com/github/ClaytonUrbano/analise-de-dados/blob/main/C%C3%B3pia_de_Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install -q pandas numpy scikit-learn plotly joblib streamlit pyngrok psutil





In [ ]:
from google.colab import files
uploaded = files.upload()  # selecione seus arquivos .csv
# Após o upload, mova para a pasta data/
import shutil, os
for fname in uploaded.keys():
    shutil.move(fname, f"data/{fname}")
os.listdir("data")


In [ ]:
import pandas as pd

df = pd.read_csv("logistics_data.csv")
print(df.head())
print(df.dtypes)

# Se houver coluna de tempo, parse:
if "timestamp" in df.columns:
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df = df.dropna(subset=["timestamp"]).sort_values("timestamp")

# Conferir valores faltantes
missing = df.isna().sum()
missing[missing>0]


In [ ]:
# Só se tiver timestamp
if "timestamp" in df.columns:
    df = df.set_index("timestamp").resample("5min").agg({
        "cpu":"mean", "ram":"mean", "latency_ms":"median", "rps":"sum"
    }).dropna().reset_index()
df.head()


In [ ]:
import plotly.express as px

for col in [c for c in ["cpu","ram","latency_ms","rps"] if c in df.columns]:
    fig = px.line(df, x="timestamp" if "timestamp" in df.columns else df.index, y=col, title=col)
    fig.show()


In [ ]:
from sklearn.ensemble import IsolationForest
import joblib
import numpy as np

# Defina as features existentes no seu dataset
features = ["Weight_kg", "Cost", "Distance_miles", "Transit_Days"]
assert len(features) > 0, "Defina a lista de features conforme seu CSV."

X = df[features].astype(float)

# Treino
iso = IsolationForest(
    n_estimators=200, contamination=0.02, random_state=42, n_jobs=-1
)
iso.fit(X)

# Scoring
df["anomaly_score"] = -iso.decision_function(X)  # maior score = mais anômalo
df["is_anomaly"] = (iso.predict(X) == -1)

# Salvar artefatos
joblib.dump(iso, "models/iso_forest.pkl")
df.to_csv("data/metrics_scored.csv", index=False)

df.tail()

In [ ]:
df.sort_values("anomaly_score", ascending=False).head(10)


In [ ]:
if "timestamp" in df.columns:
    fig = px.scatter(df, x="timestamp", y=features[0], color="is_anomaly",
                     color_discrete_map={True:"red", False:"steelblue"},
                     title=f"{features[0]} com anomalias")
    fig.show()


In [ ]:
import numpy as np

def alert_rule(row):
    hard = []
    if "cpu" in row and row["cpu"] > 90: hard.append("CPU>90%")
    if "latency_ms" in row and row["latency_ms"] > 800: hard.append("Latência>800ms")
    if "ram" in row and row["ram"] > 90: hard.append("RAM>90%")
    hard_trigger = len(hard) > 0
    model_trigger = row["anomaly_score"] > np.percentile(df["anomaly_score"], 90)
    return hard_trigger and model_trigger, ", ".join(hard) if hard else ""

df["alert"], df["alert_reason"] = zip(*df.apply(alert_rule, axis=1))
alerts = df[df["alert"]].copy()
alerts.tail(10)


In [ ]:
alerts.tail(100).to_csv("data/alerts_recent.csv", index=False)


In [ ]:
%%writefile app/streamlit_app.py
import streamlit as st, pandas as pd, plotly.express as px, numpy as np
from pathlib import Path

st.set_page_config(page_title="AIOps - Incidentes Preditivos", layout="wide")
st.title("AIOps: Detecção Preditiva de Incidentes")

path = Path("data/metrics_scored.csv")
if not path.exists():
    st.warning("Gere o arquivo metrics_scored.csv (rode a célula de Isolation Forest).")
    st.stop()

df = pd.read_csv(path, parse_dates=["timestamp"], infer_datetime_format=True)
features = [c for c in ["cpu","ram","latency_ms","rps"] if c in df.columns]

with st.sidebar:
    st.header("Controles")
    window = st.slider("Registros (mais recentes)", 500, min(5000, len(df)), min(2000, len(df)))
    pctl = st.slider("Threshold de score (percentil)", 50, 99, 90)

df_show = df.tail(window).copy()
thr = np.percentile(df["anomaly_score"], pctl)

col1, col2, col3 = st.columns(3)
col1.metric("Registros", len(df))
col2.metric("Anomalias (modelo)", int(df["is_anomaly"].sum()))
col3.metric("Threshold pctl", pctl)

if features:
    fig = px.line(df_show, x="timestamp", y=features, title="Métricas")
    st.plotly_chart(fig, use_container_width=True)

fig2 = px.line(df_show, x="timestamp", y="anomaly_score", title="Score de Anomalia")
fig2.add_hline(y=thr, line_dash="dot", line_color="red")
st.plotly_chart(fig2, use_container_width=True)

st.subheader("Alertas recentes")
if "alert" in df_show.columns:
    alerts = df_show[df_show["alert"] == True].tail(100)
    if not alerts.empty:
        st.dataframe(alerts[["timestamp"] + features + ["anomaly_score","alert_reason"]])
    else:
        st.success("Sem alertas nesta janela.")
else:
    st.info("Coluna 'alert' não encontrada. Rode a célula de regras de alerta.")


In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok

In [ ]:
from pyngrok import ngrok

# Coloque seu token aqui
NGROK_AUTH_TOKEN = "32vqRAEedyXFdxElSe0uUjEEvh3_5ThHvCDQUCmq8PuCQ5rxE "

ngrok.set_auth_token(NGROK_AUTH_TOKEN)


In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/api/hello")
def hello():
    return {"Hello": "World"}


In [ ]:
from google.colab import userdata
print(userdata.get('NGROK_AUTH_TOKEN'))

In [ ]:
from fastapi import FastAPI
import nest_asyncio
import uvicorn
from pyngrok import ngrok
from google.colab import userdata

app = FastAPI()

@app.get("/")
def home():
    return {"mensagem": "Servidor rodando com ngrok no Colab!"}

# Permite rodar o servidor no Colab
nest_asyncio.apply()

# Recupera o token salvo nos Segredos do Colab
authtoken = userdata.get("NGROK_AUTH_TOKEN")

# Configura o pyngrok para usar o token
if authtoken:
    ngrok.set_auth_token(authtoken)
    print("Authtoken do ngrok configurado.")
else:
    print("Authtoken do ngrok não encontrado nos segredos do Colab. Por favor, adicione-o usando o ícone de chave (🔑) na barra lateral esquerda com o nome 'NGROK_AUTH_TOKEN'.")
    # Stop execution if the authtoken is not found
    exit()
# Cria o app FastAPI
app = FastAPI()

# Inicia o túnel na porta 8000
public_url = ngrok.connect(8000)
print("URL pública:", public_url)

# Inicia o servidor
uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
from pyngrok import ngrok
import os
from google.colab import userdata

# Recupera o token salvo nos Segredos do Colab
authtoken = userdata.get("NGROK_AUTH_TOKEN")

# Configura o pyngrok para usar o token
if authtoken:
    ngrok.set_auth_token(authtoken)
    print("Authtoken do ngrok configurado.")
else:
    print("Authtoken do ngrok não encontrado nos segredos do Colab. Por favor, adicione-o usando o ícone de chave (🔑) na barra lateral esquerda com o nome 'NGROK_AUTH_TOKEN'.")

In [ ]:
from google.colab import userdata
userdata.get('NGROK_AUTH_TOKEN')

In [ ]:
#Para aumentar a segurança e não ter problemas em compartilhar seu notebook do colab
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
#🧰 Etapas do Painel Interativo
#✅ 1. Instalar dependências
!pip install pyngrok fastapi uvicorn nest_asyncio


In [ ]:
#📝 2. Criar e visualizar configuração JSON
import json
from IPython.display import display, Markdown

# Configuração inicial
config = {
    "agente": {
        "nome": "AgenteClayton",
        "prioridade": "alta",
        "limite_trafego": "500mbps"
    }
}

# Salvar como JSON
with open("config.json", "w") as f:
    json.dump(config, f, indent=4)

# Mostrar conteúdo atual
def mostrar_config():
    with open("config.json") as f:
        texto = f.read()
    display(Markdown(f"### Configuração atual:\n```json\n{texto}\n```"))

mostrar_config()



In [ ]:
#🧑‍💻 3. Criar função para editar a configuração
def atualizar_config(novo_nome, nova_prioridade, novo_limite):
    config["agente"]["nome"] = novo_nome
    config["agente"]["prioridade"] = nova_prioridade
    config["agente"]["limite_trafego"] = novo_limite
     with open("config.json", "w") as f:
        json.dump(config, f, indent=4)
    print("✅ Configuração atualizada.")
    mostrar_config()


In [ ]:
#Você pode chamar essa função assim:
atualizar_config("AgenteX", "média", "200mbps")


In [ ]:
#🚀 4. Iniciar servidor FastAPI com essa configuração
from fastapi import FastAPI
import nest_asyncio
import uvicorn
from pyngrok import ngrok
from google.colab import userdata # Import userdata
import json # Import the json library
import yaml # Import the yaml library

app = FastAPI()

@app.get("/config")
def ler_config():
    with open("config.json") as f: # Change filename to config.json
        return json.load(f) # Change to json.load

nest_asyncio.apply()

# Configurar ngrok
authtoken = userdata.get("NGROK_AUTH_TOKEN") # Get token from Colab secrets
if authtoken:
    ngrok.set_auth_token(authtoken)
    print("Authtoken do ngrok configurado.")
else:
    print("Authtoken do ngrok não encontrado nos segredos do Colab. Por favor, adicione-o usando o ícone de chave (🔑) na barra lateral esquerda com o nome 'NGROK_AUTH_TOKEN'.")
    # You might want to handle this case, e.g., by exiting or raising an error
    # For now, we'll just print a message and continue, but ngrok.connect will likely fail
    pass # Or sys.exit(1) or raise ValueError(...)


# Iniciar túnel
public_url = ngrok.connect(8000)
print("🔗 Interface disponível em:", public_url)

# Iniciar servidor
uvicorn.run(app, host="0.0.0.0", port=8000)

Authtoken do ngrok configurado.
🔗 Interface disponível em: NgrokTunnel: "https://3707f59e3d07.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [226]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2804:a74:322:1e00:7902:dfd1:7464:fb6a:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2804:a74:322:1e00:7902:dfd1:7464:fb6a:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
